In [ ]:
from nbtemplate import display_header
display_header('index.ipynb')

# Ray-tracing the Lynx XGS

[Lynx](https://www.lynxobservatory.com/) is one of the four large surveyors that NASA looked at through detailed science and technical studies in preparation for the 2020 Decadal survey. In the Design Reference Mission (DRM) Lynx has a diffraction grating for high-resolution spectrocopy of soft X-rays. The science requirements for this instrument are a spectral resolving power $R > 5000$ and an effective area $A_{\mathrm{eff}} > 4000$ cm$^2$ at 0.6 keV. Such a spectrometer could be build with either reflection or transmission gratings. The DRM baselines critical angle transmission (CAT) gratings. On these pages, I present ray-traces for performance and trades for the CAT XGS. Most of the content is published in the following literature references, but the results presented here have been updated with respect to that and include additional material, such as more detailed plots and interactive 3D models of the instrument:

- [Lynx grating spectrometer design: Optimizing chirped transmission gratings (SPIE proc, 2020)](https://ui.adsabs.harvard.edu/abs/2020arXiv201114862G/abstract)
- [Design progress on the Lynx soft x-ray critical-angle transmission grating spectrometer (SPIE proc, 2019)](https://ui.adsabs.harvard.edu/abs/2020arXiv201114862G/abstract)
- [Lynx soft x-ray critical-angle transmission grating spectrometer (JATIS, 2019)](https://ui.adsabs.harvard.edu/abs/2019JATIS...5b1003G/abstract)

Please let me know if you have any questions or need high-resolution versions of any of the plots or
the data to recreate the plot by emailing hgunther@mit.edu . (The plots here are optimized for viewing on the web and do not have enough resolution for print publications.)

Here is the pointer to the software used:

- My general ray-trace code [MARXS](http://marxs.readthedocs.io/en/latest/index.html)
- Ray-trace code [specific to Lynx](https://github.com/hamogu/marxs-lynx)

## Lynx XGS: How does is look?
In this section, I show how the XGS setup looks. I start with an interactive 3D view, which can be zoomed and rotated with the mouse in all supported browsers; pressing "r" returns the view to the initial position. See [the X3DOM documentation](https://www.x3dom.org/documentation/interaction/) for a full list of supported mouse and keyboard commands.

Rays start in the aperture, which is shaped like a series of concentrig rings. The exact dimensions of the mirror, in particular the position of support spiders etc. are somewhat arbitrary at this point, since the mirror design is still nominal and will be refined at a later stage in the development. In the Lynx mirrors, photons will bounce twice in a Wolter-Schwarzschild geometry. However, in this simulation the mirror is simplified such that the reflection actually happens in a single plane, shown in white (the the double refelction is taken into account when calcualting the survoval probability of a photon).  


Rays are imaged onto detectors (yellow). The color of the rays is according to their diffraction order. Since this is a monoenergetic simualtion, the orders are well separated.

In [ ]:
from mayavi import mlab
mlab.init_notebook('x3d', 800, 500, local=False)

In [ ]:
import numpy as np
import astropy.units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord
from marxs.source import PointSource, FixedPointing
from marxs import simulator
from marxs.visualization.mayavi import plot_object, plot_rays
from marxslynx.ralfgrating import facet_table
from marxslynx import lynx

In [ ]:
n_photons = 1e4
wave = np.arange(6., 60., 0.5) * u.Angstrom
energies = wave.to(u.keV, equivalencies=u.spectral())
instrument = lynx.LynxForPlot(conf=lynx.conf_chirp)


In [ ]:
facettab = facet_table(instrument.elements[2])

In [ ]:
ind = np.abs(np.abs(facettab['facet_ang'])) < np.pi * 2 / 3

In [ ]:
instrument.elements[2].elements = [instrument.elements[2].elements[i] for i in ind.nonzero()[0]]

In [ ]:
src = PointSource(coords=SkyCoord(30. * u.deg, 30. * u.deg),
                           energy=0.4 * u.keV,
                           flux=1. / u.s / u.cm**2)
pointing = FixedPointing(coords=SkyCoord(30 * u.deg, 30. * u.deg))

In [ ]:
keeppos = simulator.KeepCol('pos')
keepprob = simulator.KeepCol('probability')
instrument.postprocess_steps = [keeppos, keepprob]
photons = src.generate_photons(n_photons * u.s)
photons = pointing(photons)
photons = instrument(photons)

In [ ]:
fig = mlab.figure(bgcolor=(.8, .8, .8))
mlab.clf()

out = plot_object(instrument, viewer=fig)

pos = keeppos.format_positions()
ind = (photons['CCD_ID'] >= 0) | np.isfinite(photons['microcal_x'])
photons['order'][np.isnan(photons['order'])] = 0
out = plot_rays(pos[ind, :, :], scalar=photons['order'][ind], viewer=fig)


# move camera programatically mlab.view()
fig

Basic design for a Lynx CAT grating spectrometer, showing a ray-trace for an on-axis point source with a monochromatic emission at 0.4 keV = 31 Å.
This is the scenario with the largest gratings we studied (60 mm * 180 mm). Gratings are arbitrarily colored in sectors. Not the entire aperture is filled with gratings. We study different ways to sub-aperture and this is just one possible configuration.

Rays detected in the zeroth order (direct light) are shown in grey, diffracted photons in order -3 in green. The other orders do not fall onto detectors and are thus not shown.